In [2]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

I0000 00:00:1730490766.829527     244 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730490766.952298     244 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730490766.952436     244 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
import polars as pl
import numpy as np
import os
directory = '/app/finalDataSets'
# directory = r'D:\github\localt20'

playerStats = tf.convert_to_tensor(np.load(os.path.join(directory,'playerStats.npy')))
balltoball = tf.convert_to_tensor(pl.read_csv(os.path.join(directory,'balltoball.csv')))
teamStats = tf.convert_to_tensor(pl.read_csv(os.path.join(directory,'team12Stats.csv')))

# Shapes of the data
playerStats_shape = playerStats.shape
teamStats_shape = teamStats.shape
balltoball_shape = balltoball.shape
playerStats_shape,teamStats_shape,balltoball_shape

(TensorShape([1156, 22, 23]),
 TensorShape([1156, 24]),
 TensorShape([267455, 6]))

In [13]:
tfplayerStats = tf.data.Dataset.from_tensor_slices(playerStats).batch(32).prefetch(tf.data.AUTOTUNE)
tfteamStats = tf.data.Dataset.from_tensor_slices(teamStats).batch(1).prefetch(tf.data.AUTOTUNE)
tfplayerStats

<_PrefetchDataset element_spec=TensorSpec(shape=(None, 22, 23), dtype=tf.float64, name=None)>

In [20]:
balltoball.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [29]:
balltoball = balltoball.numpy()
balltoball[balltoball[:, 0].argsort()]

array([[2.110280e+05, 1.000000e+00, 1.000000e-01, 0.000000e+00,
        0.000000e+00, 1.000000e+00],
       [2.110280e+05, 2.000000e+00, 2.100000e+00, 2.000000e+01,
        0.000000e+00, 0.000000e+00],
       [2.110280e+05, 2.000000e+00, 2.200000e+00, 2.100000e+01,
        0.000000e+00, 0.000000e+00],
       ...,
       [1.450765e+06, 1.000000e+00, 1.450000e+01, 7.500000e+01,
        7.000000e+00, 1.000000e+00],
       [1.450765e+06, 1.000000e+00, 1.240000e+01, 6.300000e+01,
        7.000000e+00, 1.000000e+00],
       [1.450765e+06, 1.000000e+00, 1.530000e+01, 7.900000e+01,
        7.000000e+00, 1.000000e+00]])

In [34]:
balltoball[balltoball[:, 0].argsort()].shape

(267455, 6)

In [35]:
pl.DataFrame(balltoball[balltoball[:, 0].argsort()])

column_0,column_1,column_2,column_3,column_4,column_5
f64,f64,f64,f64,f64,f64
211028.0,1.0,0.1,0.0,0.0,1.0
211028.0,2.0,2.1,20.0,0.0,0.0
211028.0,2.0,2.2,21.0,0.0,0.0
211028.0,2.0,2.3,23.0,0.0,0.0
211028.0,2.0,2.4,25.0,0.0,0.0
…,…,…,…,…,…
1.450765e6,1.0,14.3,71.0,7.0,1.0
1.450765e6,1.0,14.4,75.0,7.0,1.0
1.450765e6,1.0,14.5,75.0,7.0,1.0


In [36]:
pl.DataFrame(balltoball)

column_0,column_1,column_2,column_3,column_4,column_5
f64,f64,f64,f64,f64,f64
1.001349e6,1.0,0.1,0.0,0.0,0.0
1.001349e6,1.0,0.2,0.0,0.0,0.0
1.001349e6,1.0,0.3,1.0,0.0,0.0
1.001349e6,1.0,0.4,3.0,0.0,0.0
1.001349e6,1.0,0.5,3.0,0.0,0.0
…,…,…,…,…,…
995469.0,2.0,17.1,126.0,6.0,1.0
995469.0,2.0,17.2,129.0,6.0,1.0
995469.0,2.0,17.3,129.0,6.0,1.0
